In [1]:
import numpy as np
import networkx as nx
from SNN_neat import DAG, plot_dag, Layer
from SNN_neat import ReLU, LeakyReLU, Sigmoid, Linear, Tanh
from tqdm import tqdm
import plotly.graph_objects as go
from time import time

In [2]:
WIDTH = 10
NET_DEPH = 20

dag = DAG(WIDTH, WIDTH)

for i, node_id in enumerate(dag.input_nodes):
    out_node = dag.output_nodes[i]
    dag.add_connection(node_id, out_node)
    
    # add node in input connection
    new_node = node_id
    for i in range(NET_DEPH):
        _, new_node = dag.add_node(new_node, out_node)

dag.processing_order = dag.get_processing_order()
for i, layer in enumerate(dag.processing_order[:-1]):
    for node1 in layer:
        for node2 in dag.processing_order[i+1]:
            dag.add_connection(node1, node2)

dag.processing_order = dag.get_processing_order() 

In [3]:
def calculate_correlation(layer1: Layer, layer2: Layer, dag: DAG):
    a1 = []
    for node in layer1:
        activity = dag.get_node(node).activity
        activity = np.where(activity == 0, -1, activity)
        a1.append(np.average(activity))

    a2 = []
    for node in layer2:
        activity = dag.get_node(node).activity
        activity = np.where(activity == 0, -1, activity)
        a2.append(np.average(activity))

    return np.abs(np.average((a1 - np.average(a1)) * (a2 - np.average(a2))))

In [4]:
THRESHOLD = 0.5
ITERATIONS = 100

results = {}

densities = []
for i in range(ITERATIONS):
    input_density = np.random.rand() * 0.6 + 0.1
    densities.append([int(np.random.rand() < input_density) for _ in range(WIDTH)])

activity_results = {i: [[] for _ in range(len(densities))] for i in range(len(dag.processing_order))}
activity_results[0] = np.average(densities, axis=1)


for eb in tqdm(np.arange(0, 1.5, 0.05)):
    results[eb] = []
    # iterate multipple times
    for i, d in enumerate(densities):
        dag.reset()
        inputs = d

        for layer in dag.processing_order:
            for nodeId in layer:
                node = dag.get_node(nodeId)
                energy = (np.random.rand() * 2 - 1) * eb # <- energy bias 
                node.weights = np.array([ (np.random.rand() * 2 - 1) for _ in range(WIDTH) ]) * energy
                node.bias = 0 #1 / (WIDTH + 1) # + energy
                node.threshold = THRESHOLD #+ energy

        # process the network #####
        dag.process_once(inputs)
        ###########################

        # get layer activities
        for l, layer in enumerate(dag.processing_order[1:], 1):
            la = []
            for node in layer:
                la.append(np.average(dag.get_node(node).activity))
                # index layer, index density number, append the average activity
            activity_results[l][i].append(la)

 
    # calculate correlations for all nodes to the node in the layer before
    correlations = []
    input_node_id = dag.input_nodes[0]
    for i, layer in enumerate(dag.processing_order[1:]):
        correlation = calculate_correlation(dag.processing_order[0], layer, dag)
        results[eb].append(correlation)

# calculate average activity for each layer for each density
for i, layer in enumerate(dag.processing_order[1:], 1):
    for j in range(len(densities)):
        activity_results[i][j] = np.average(activity_results[i][j])
        

print(np.array(activity_results[0]).shape,np.array(activity_results[1]).shape)

100%|██████████| 30/30 [00:16<00:00,  1.77it/s]

(100,) (100,)


In [5]:
# plot all the tested thresholds, by different lines in the xs vs. ys plot with plotly
# x values are input activity, y values are mean activity of the second layer
fig = go.Figure()
for eb, result in results.items():
    fig.add_trace(go.Scatter(x=list(range(1, len(result)+1)), y=result, mode='lines+markers', name="eb: " + str(eb)))

fig.update_xaxes(title="Layer")
fig.update_yaxes(title="correlation")
fig.show()


In [6]:
# get correlation length
correlation_length = np.zeros(len(results))
all_ebs = list(results.keys())
all_ebs.sort()
for j, eb in enumerate(all_ebs):
    # find 0 crossing
    for i, r in enumerate(results[eb]):
        if r <= 0:
            correlation_length[j] = i+1
            break

# plot corelation length vs. eb
fig = go.Figure()
fig.add_trace(go.Scatter(x=all_ebs, y=correlation_length, mode='lines'))
fig.update_xaxes(title="Energy Bias")
fig.update_yaxes(title="correlation length")
fig.show()

In [7]:
# plot activity for each density vs layer activity
fig = go.Figure()
for i, layer in enumerate(dag.processing_order[1:], 1):
    fig.add_trace(go.Scatter(x=activity_results[0], y=activity_results[i], mode='markers', name="Layer: " + str(i)))

fig.update_xaxes(title="Input Activity")
fig.update_yaxes(title="Layer Activity")
fig.show()